In [1]:
import requests
import pandas as pd
import pandas_ta as ta
import numpy as np
import alphalens
from tabulate import tabulate
import plotly.graph_objects as go
import time
import pytz
from datetime import datetime
import json

In [2]:
target_list = ['BTCUSDT',
 'ETHUSDT',
 'BNBUSDT',
 'AAVEUSDT',
 'AVAXUSDT',
 'ETCUSDT',
 'SOLUSDT',
 'ADAUSDT',
 'LINKUSDT',
 'BCHUSDT',
 'DOTUSDT',
 'LTCUSDT',
 'UNIUSDT',
 'XRPUSDT',
 'XLMUSDT',
 'MATICUSDT',
 'TRXUSDT',
 'DOGEUSDT',
 'INJUSDT',
 'DASHUSDT',
 'XEMUSDT',
 'ZECUSDT']
starttime = '2022/01/01 00:00:00'
endtime = '2024/04/05 00:00:00'
target_timezone = pytz.timezone("Asia/Hong_Kong")   # target_timezone = 'UTC'
interval = "1d"

In [3]:
dataframes_dict = {}  # Initialize an empty dictionary

for target in target_list:

    pair = target
    print(pair)
    ContractType = "PERPETUAL"
    start_time = int(pd.Timestamp(starttime, tz=target_timezone).timestamp() * 1000)
    end_time = int(pd.Timestamp(endtime, tz=target_timezone).timestamp() * 1000)
    limit = 1000

    all_data = []

    # Make requests in chunks until you reach the end_time
    while start_time < end_time:
        params = {
            "pair": pair,
            "ContractType": ContractType,
            "interval": interval,
            "startTime": start_time,
            "endTime": end_time,
            "limit": limit  
        }
        
        url = "https://fapi.binance.com/fapi/v1/continuousKlines"
        response = requests.get(url, params=params)
        data = response.json()
        
        if not data:
            break

        all_data.extend(data)

        # Update the start_time for the next request
        start_time = int(data[-1][0]) + 1  
        time.sleep(1)  # Be respectful with the number of requests sent to the server
    
    # Processing data after collecting for each target
    if all_data:
        df = pd.DataFrame({
            'date': [row[0] for row in all_data],
            'open': [row[1] for row in all_data],
            'high': [row[2] for row in all_data],
            'low': [row[3] for row in all_data],
            'close': [row[4] for row in all_data],
            'volume': [row[5] for row in all_data],
            #'datetime': [row[6] for row in all_data],
            'volvalue': [row[7] for row in all_data],
            'takerbuy': [row[8] for row in all_data],
            'takerbuyvalue': [row[9] for row in all_data],
        })

        df['date'] = pd.to_datetime(df['date'], unit='ms')
        df['date'] = df['date'].dt.strftime('%Y-%m-%d %H:%M:%S')

        # Save DataFrame in dictionary
        dataframes_dict[target] = df



BTCUSDT
ETHUSDT
BNBUSDT
AAVEUSDT
AVAXUSDT
ETCUSDT
SOLUSDT
ADAUSDT
LINKUSDT
BCHUSDT
DOTUSDT
LTCUSDT
UNIUSDT
XRPUSDT
XLMUSDT
MATICUSDT
TRXUSDT
DOGEUSDT
INJUSDT
DASHUSDT
XEMUSDT
ZECUSDT


In [4]:
BTC = dataframes_dict['BTCUSDT'].set_index('date')
ETH = dataframes_dict['ETHUSDT'].set_index('date')
BNB = dataframes_dict['BNBUSDT'].set_index('date')
AAVE = dataframes_dict['AAVEUSDT'].set_index('date')
AVAX = dataframes_dict['AVAXUSDT'].set_index('date')
ETC = dataframes_dict['ETCUSDT'].set_index('date')
SOL = dataframes_dict['SOLUSDT'].set_index('date')
ADA = dataframes_dict['ADAUSDT'].set_index('date')
LINK = dataframes_dict['LINKUSDT'].set_index('date')
BCH = dataframes_dict['BCHUSDT'].set_index('date')
DOT = dataframes_dict['DOTUSDT'].set_index('date')
LTC = dataframes_dict['LTCUSDT'].set_index('date')
UNI = dataframes_dict['UNIUSDT'].set_index('date')
XRP = dataframes_dict['XRPUSDT'].set_index('date')
XLM = dataframes_dict['XLMUSDT'].set_index('date')
MATIC = dataframes_dict['MATICUSDT'].set_index('date')
TRX = dataframes_dict['TRXUSDT'].set_index('date')
DOGE = dataframes_dict['DOGEUSDT'].set_index('date')
INJ = dataframes_dict['INJUSDT'].set_index('date')
DASH = dataframes_dict['DASHUSDT'].set_index('date')
XEM = dataframes_dict['XEMUSDT'].set_index('date')
ZEC = dataframes_dict['ZECUSDT'].set_index('date')

In [5]:
target_list = [
    BTC, ETH, BNB, AAVE, AVAX, ETC, SOL, ADA, LINK,
    BCH, DOT, LTC, UNI, XRP, XLM, MATIC, TRX, DOGE, INJ, DASH, XEM, ZEC
]
target_str_list =  [
    "BTC", "ETH", "BNB", "AAVE", "AVAX", "ETC", 
    "SOL", "ADA", "LINK", "BCH", "DOT", "LTC", 
    "UNI", "XRP", "XLM", "MATIC", "TRX", "DOGE", 
    "INJ", "DASH", "XEM", "ZEC"
]
column_list = ['open', 'high', 'low', 'close', 'volume', 'volvalue', 'takerbuy', 'takerbuyvalue']

column_dfs = {}

for column in column_list:
    series_list = []
    for target, name in zip(target_list, target_str_list):
        selected = target[column].copy()  
        selected.name = name
        series_list.append(selected)

    column_df = pd.concat(series_list, axis=1)
    column_dfs[column] = column_df

combined_df = pd.concat(column_dfs, axis=1)

In [6]:
### There are the dataframe combine and arrange by factor
combined_df = combined_df.apply(lambda x: pd.to_numeric(x, errors='coerce'))
combined_df.index = pd.to_datetime(combined_df.index)
combined_df

open                                                     \
                 BTC      ETH     BNB    AAVE     AVAX     ETC      SOL   
date                                                                      
2022-01-01  46210.57  3676.01  511.55  253.95  109.441  34.057  170.010   
2022-01-02  47704.35  3763.05  527.36  266.35  114.079  34.861  179.060   
2022-01-03  47280.01  3826.10  530.92  265.29  113.060  35.370  176.270   
2022-01-04  46445.80  3764.07  512.24  267.31  108.105  34.709  170.200   
2022-01-05  45833.41  3785.60  506.88  251.93  103.351  34.120  167.870   
...              ...      ...     ...     ...      ...     ...      ...   
2024-03-31  69642.00  3508.66  600.26  124.04   52.980  32.927  194.351   
2024-04-01  71363.00  3648.80  606.04  128.47   54.135  34.313  202.685   
2024-04-02  69700.00  3506.52  576.41  129.50   51.435  32.714  192.355   
2024-04-03  65502.40  3280.67  551.84  115.55   46.902  30.008  181.671   
2024-04-04  65971.80  3313.28  562.13  114.59   45.913  30.657  185.179   

                                    ... takerbuyvalue                          \
               ADA    LINK     BCH  ...           UNI          XRP        XLM   
date                                ...                                         
2022-01-01  1.3077  19.514  430.23  ...       2377138  163804665.7  103404616   
2022-01-02  1.3781  20.738  444.74  ...       2652291  160532067.6  140813237   
2022-01-03  1.3767  21.852  447.52  ...       5792958  201179957.4  192400959   
2022-01-04  1.3196  23.699  434.78  ...       4141985  229296946.9  155115064   
2022-01-05  1.3093  23.426  426.84  ...       6713635  399855414.4  262661549   
...            ...     ...     ...  ...           ...          ...        ...   
2024-03-31  0.6442  18.953  598.16  ...       4421646  201977255.7   74956444   
2024-04-01  0.6507  19.193  679.59  ...       5592509  605739931.6  232606977   
2024-04-02  0.6218  18.378  648.33  ...       6408919  862676284.8  261761157   
2024-04-03  0.5810  17.970  639.75  ...       6855572  556485097.9  225905531   
2024-04-04  0.5697  17.697  593.07  ...       3227909  932743039.9  213594485   

                                                                     \
                MATIC        TRX        DOGE        INJ        DASH   
date                                                                  
2022-01-01   85390204  317796814   454318217        NaN   90366.138   
2022-01-02   70552280  348535420   512945767        NaN   82528.760   
2022-01-03   90205852  435109520   690618711        NaN   87966.974   
2022-01-04  103796661  440046024   737315588        NaN  109611.227   
2022-01-05  143067300  754419967  1285955755        NaN  187244.854   
...               ...        ...         ...        ...         ...   
2024-03-31   66158229  119779857  3840515749  1283725.9   93399.138   
2024-04-01  158312032  242205239  5571652953  2787211.1  189956.207   
2024-04-02  181522927  346226893  6607709765  2797750.3  193532.777   
2024-04-03  132503332  302798562  5513410622  3102754.3  146058.593   
2024-04-04  112338078  208790862  4195728729  1869975.9  172172.162   

                                   
                  XEM         ZEC  
date                               
2022-01-01   34129219  140576.537  
2022-01-02   53810895  161578.162  
2022-01-03   55002070  134912.426  
2022-01-04   53272400  237060.845  
2022-01-05  333081952  271332.872  
...               ...         ...  
2024-03-31  124655640  199828.914  
2024-04-01  297945584  430081.705  
2024-04-02  452544937  370889.693  
2024-04-03  261136533  321819.407  
2024-04-04  211241848  295169.723  

[825 rows x 176 columns]

In [7]:
Open = combined_df['open']
High = combined_df['high']
Low = combined_df['low']
Close = combined_df['close']
returns = Close.pct_change()
Volume = combined_df['volume']
Volume_v = combined_df['volvalue']
takerbuy = combined_df['takerbuy']
takerbuy_v = combined_df['takerbuyvalue']
exp_returns = Close.pct_change().shift(-2)
benchmark = exp_returns['BTC']
BTC_index = returns['BTC']


def get_cumprod_returns(data):
    ret_cum = (1 + data).cumprod() - 1
    return ret_cum.iloc[-1]

def get_cumsum_returns(data):
    ret_cum = data.cumsum()
    return ret_cum.iloc[-1]

def get_sharpe(data, period_of_year):
    Sharpe_ratio = data.mean() / data.std() * np.sqrt(period_of_year)
    return Sharpe_ratio

def get_volatility(data, period_of_year):
    annual_vol = data.std() * np.sqrt(period_of_year)
    return annual_vol

def get_std(data):
    return data.std()

def get_annual_returns(data, period_of_year):
    compound = (data + 1).cumprod()
    days = len(compound)
    total_return = compound.iloc[-1] - 1
    annual_factor = period_of_year
    annualized_return = (total_return + 1) ** (annual_factor / days) - 1
    return annualized_return

def get_turnover(weights):
    delta_weight = weights.diff()
    daily_trading_value = delta_weight.abs().sum(axis=1)
    turnover = daily_trading_value.sum() / len(daily_trading_value)
    return turnover

def get_mdd(data):
    compound = (data + 1).cumprod()
    drawdowns = compound / compound.cummax() - 1
    max_drawdown = drawdowns.min()
    return abs(max_drawdown)

def get_performance_report(returns_by_period, benchmark, period_of_year):
    returns_by_period = returns_by_period.dropna()
    benchmark = benchmark.dropna()
    returns_by_period_first_index = returns_by_period.index[0]
    benchmark_first_index = benchmark.index[0]
    returns_by_period_last_index = returns_by_period.index[-1]
    benchmark_last_index = benchmark.index[-1]

    common_start_date = max(returns_by_period_first_index, benchmark_first_index)
    common_end_date = min(returns_by_period_last_index, benchmark_last_index)
    returns_by_period = returns_by_period[common_start_date:common_end_date]
    benchmark = benchmark.loc[common_start_date:common_end_date]  
    
    summary_df = pd.DataFrame({
        'Cumprod Total Returns': [f"{get_cumprod_returns(returns_by_period) * 100:.2f} %",
                                    f"{get_cumprod_returns(benchmark) * 100:.2f} %"],
        'Cumsum Total Returns': [f"{get_cumsum_returns(returns_by_period) * 100:.2f} %",
                                    f"{get_cumsum_returns(benchmark) * 100:.2f} %"],
        'Sharpe Ratio': [f"{get_sharpe(returns_by_period, period_of_year):.2f}",
                            f"{get_sharpe(benchmark, period_of_year):.2f}"],
        'Annualized Ret': [f"{get_annual_returns(returns_by_period, period_of_year) * 100:.2f} %",
                            f"{get_annual_returns(benchmark, period_of_year) * 100:.2f} %"],
        'Max Drawdown': [f"{get_mdd(returns_by_period) * 100:.2f} %",
                            f"{get_mdd(benchmark) * 100:.2f} %"],
        'Volatility': [f"{get_volatility(returns_by_period, period_of_year) * 100:.2f} %",
                        f"{get_volatility(benchmark, period_of_year) * 100:.2f} %"],
        'STD': [f"{get_std(returns_by_period) * 100:.2f} %",
                f"{get_std(benchmark) * 100:.2f} %"]
    }, index=['Performance', 'Benchmark'])

    return summary_df

def get_pnl(returns_by_period, benchmark):

    start_time = returns_by_period.index[0]
    end_time = returns_by_period.index[-1]
    benchmark_returns_filtered = benchmark.loc[start_time:end_time]
    benchmark_cumulative_returns = (1 + benchmark_returns_filtered).cumprod() - 1
    cumulative_returns = (1 + returns_by_period).cumprod() - 1

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=cumulative_returns.index, y=cumulative_returns, mode='lines', name='Cumulative Returns'))
    fig.add_trace(go.Scatter(x=benchmark_cumulative_returns.index, y=benchmark_cumulative_returns, mode='lines', name='Benchmark', line=dict(color='#FFA500')))

    fig.update_layout(
        title='Cumulative Returns Over Time',
        xaxis_title='Date',
        yaxis_title='Cumulative Returns',
        width=750,
        height=450  
    )
    fig.show()

def get_returns_by_period_for_plateau(factor: pd.DataFrame, expectedreturn: pd.DataFrame, period_of_year: int, fee: float = 0.0004, benchmark=None):

    demean = factor.sub(factor.mean(axis=1), axis=0)
    weights = demean.div(demean.abs().sum(axis=1), axis=0)

    # delta_weight = weights.shift(1) - weights
    delta_weight = weights.diff().abs()
    fee_by_period = delta_weight * fee
    total_fee_by_period = fee_by_period.sum(axis=1)
    profit_by_period = (weights * expectedreturn).sum(axis=1)
    returns_by_period = profit_by_period - total_fee_by_period
    returns_by_period = returns_by_period.dropna()

    return returns_by_period

def get_returns_by_period(factor: pd.DataFrame, expectedreturn: pd.DataFrame, period_of_year: int, fee: float = 0.0004, benchmark=None):

    demean = factor.sub(factor.mean(axis=1), axis=0)
    weights = demean.div(demean.abs().sum(axis=1), axis=0)

    # delta_weight = weights.shift(1) - weights
    delta_weight = weights.diff().abs()
    fee_by_period = delta_weight * fee
    total_fee_by_period = fee_by_period.sum(axis=1)
    profit_by_period = (weights * expectedreturn).sum(axis=1)
    returns_by_period = profit_by_period - total_fee_by_period
    returns_by_period = returns_by_period.dropna()

    summary_df = get_performance_report(returns_by_period, benchmark, period_of_year)
    print(tabulate(summary_df, headers='keys', tablefmt='pretty', showindex=True))
    get_pnl(returns_by_period, benchmark)

    return returns_by_period, summary_df

In [8]:
# sharpe factor
period = 21
rolling_returns = returns.rolling(window = period).mean()
rolling_std_dev = returns.rolling(window = period).std()
sharpe_factor = rolling_returns / rolling_std_dev

# sum_returns_factor
sum_ret_factor = returns.rolling(window=20).sum()

# bias factor
def ts_sma(df: pd.DataFrame, d: int):
    sma = df.apply(lambda x: ta.sma(x, length=d), axis=0)
    return sma

def ts_ema(df: pd.DataFrame, d: int):
    ema = df.apply(lambda x: ta.ema(x, length=d), axis=0)
    return ema

def ts_wma(df: pd.DataFrame, d: int) -> pd.DataFrame:
    weights = np.arange(1, d + 1)
    
    def wma(x):
        return np.dot(x, weights) / weights.sum()
    wma_df = df.rolling(window=d, min_periods=d).apply(wma, raw=True)
    
    return wma_df
def ts_bias(df, d, method='sma'):
    if method == 'sma':
        ma = ts_sma(df, d)
    elif method == 'ema':
        ma = ts_ema(df, d)
    elif method == 'wma':
        ma = ts_wma(df, d)
    else:
        raise ValueError("Invalid method. Please use 'sma', 'ema', or 'wma'.")

    bias = (df - ma) / ma * 100
    return bias

bias_factor = ts_bias(df = Close, d = 27, method='sma')


# cmo factor
def ts_cmo(df: pd.DataFrame, d: int) -> pd.DataFrame:

    delta = df.diff()
    
    up = delta.clip(lower=0) 
    down = -delta.clip(upper=0)  

    sum_up = up.rolling(window=d, min_periods=d).sum()
    sum_down = down.rolling(window=d, min_periods=d).sum()

    cmo = (sum_up - sum_down) / (sum_up + sum_down) * 100
    return cmo

cmo_factor = ts_cmo(Close, 20)

# combine together and only give signal to quantile five and one
Multi_alpha = sharpe_factor + sum_ret_factor + bias_factor + cmo_factor
Multi_alpha = Multi_alpha.dropna()
ranked_df = Multi_alpha.dropna().rank(axis=1, pct=True)
signal_df = ranked_df.where(~((ranked_df > 0.2) & (ranked_df < 0.8)), np.nan)

In [9]:
print("start time:", Multi_alpha.index[0],"end time:", Multi_alpha.index[-1])
returns_by_period,summary_df = get_returns_by_period(Multi_alpha, exp_returns, 365, 0.0004, benchmark)

start time: 2022-09-12 00:00:00 end time: 2024-04-04 00:00:00
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+
| Performance |        59.00 %        |       50.49 %        |     1.41     |    34.64 %     |   18.01 %    |  23.01 %   | 1.20 % |
|  Benchmark  |       239.69 %        |       140.79 %       |     1.85     |    119.12 %    |   25.91 %    |  48.69 %   | 2.55 % |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+


In [10]:
# strategy = returns_by_period*0.7 + benchmark*0.3
# strategy = strategy.dropna()
# print("start time:", strategy.index[0],"end time:", strategy.index[-1])
# summary_df = get_performance_report(strategy, benchmark, 365)
# print(tabulate(summary_df, headers='keys', tablefmt='pretty', showindex=True))
# get_pnl(strategy,benchmark)

start time: 2022-09-12 00:00:00 end time: 2024-04-02 00:00:00
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+
|             | Cumprod Total Returns | Cumsum Total Returns | Sharpe Ratio | Annualized Ret | Max Drawdown | Volatility |  STD   |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+
| Performance |       108.46 %        |       77.58 %        |     2.17     |    60.19 %     |   16.90 %    |  22.96 %   | 1.20 % |
|  Benchmark  |       239.69 %        |       140.79 %       |     1.85     |    119.12 %    |   25.91 %    |  48.69 %   | 2.55 % |
+-------------+-----------------------+----------------------+--------------+----------------+--------------+------------+--------+


In [12]:
demean = signal_df.sub(signal_df.mean(axis=1), axis=0)
weighting = demean.div(demean.abs().sum(axis=1), axis=0)
weighting 
# factor_weighting = weighting*0.7

# btc_weighting = pd.DataFrame(0.3, index=factor_weighting.index, columns=['BTCUSDT_SPOT'])

# pool_weighting = pd.concat([factor_weighting, btc_weighting], axis=1)
# pool_weighting

,BTC,ETH,BNB,AAVE,AVAX,ETC,SOL,ADA,LINK,BCH,...,UNI,XRP,XLM,MATIC,TRX,DOGE,INJ,DASH,XEM,ZEC
date,,,,,,,,,,,,,,,,,,,,,
2022-09-12,NaN,NaN,NaN,NaN,-0.131429,0.112857,0.100000,0.074286,0.087143,NaN,...,NaN,NaN,NaN,0.125714,-0.105714,-0.144286,NaN,NaN,NaN,-0.118571
2022-09-13,NaN,NaN,NaN,NaN,-0.144286,0.100000,NaN,0.087143,0.074286,NaN,...,-0.105714,NaN,NaN,0.112857,NaN,-0.131429,NaN,NaN,NaN,-0.118571
2022-09-14,NaN,NaN,-0.105714,NaN,-0.144286,0.125714,NaN,0.074286,0.100000,NaN,...,NaN,NaN,NaN,0.087143,NaN,-0.131429,NaN,NaN,-0.118571,NaN
2022-09-15,NaN,NaN,NaN,-0.105714,-0.144286,0.074286,NaN,0.112857,0.125714,NaN,...,NaN,NaN,NaN,0.100000,NaN,-0.131429,NaN,NaN,-0.118571,NaN
2022-09-16,NaN,-0.105714,NaN,NaN,-0.144286,NaN,NaN,0.087143,0.125714,NaN,...,NaN,0.112857,NaN,NaN,NaN,NaN,0.100000,NaN,-0.118571,-0.131429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-03-31,NaN,NaN,0.087143,NaN,NaN,NaN,0.112857,-0.131429,NaN,0.125714,...,NaN,-0.105714,NaN,-0.144286,-0.118571,0.100000,NaN,NaN,NaN,NaN
2024-04-01,NaN,NaN,0.087143,NaN,NaN,NaN,0.100000,-0.118571,NaN,0.125714,...,NaN,NaN,NaN,-0.131429,NaN,0.112857,-0.144286,NaN,NaN,-0.105714
2024-04-02,0.074286,NaN,NaN,NaN,NaN,NaN,0.100000,-0.118571,NaN,0.125714,...,NaN,NaN,NaN,-0.144286,NaN,0.087143,-0.131429,NaN,NaN,NaN
